In [4]:
import ee
import datetime
import os
import itertools
import sys
import urllib.request

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [5]:
ee.Initialize()

##### Define basic parameters

In [6]:
region_cn = '东北'
region_en = 'dongbei'

In [7]:
grid_shp  = ee.FeatureCollection(f"users/wangjinzhulala/China_built_up/01_Boundary_shp/Grid_{region_en}")
sample_path = 'users/rlyon2/China_built_classification/00_sample_ext_img'

In [8]:
# define the gee-asset path for exporting
export_path = 'users/wangjinzhulala/China_built_up/03_sample_train_test_split'

In [9]:
# define the year range
year_start = [i for i in range(1990,2018,3)]
year_end   = [i for i in range(1992,2020,3)]

year_name = [f'{i[0]}_{i[1]}' for i in zip(year_start,year_end)]

In [10]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Using grid to extract sample points

In [11]:
# Define a spatial filter as geometries that intersect.
spatialFilter = ee.Filter.intersects(
                                      leftField= '.geo',
                                      rightField= '.geo',
                                      maxError= 1
                                    )

In [12]:
# Define a save all join.
saveAllJoin = ee.Join.saveAll(matchesKey= 'sample_pts')

In [15]:
for year in year_name:
    
    # get sample
    sample_pt = ee.FeatureCollection(f"{sample_path}/Control_sample_ext_img_{region_en}_{year}")
    
    #_____________________________1: select one point from each grid________________________
    # Apply the join.
    intersectJoined = saveAllJoin.apply(grid_shp, sample_pt, spatialFilter)
    
    # select one point from each grid
    choose_one = intersectJoined.map(lambda fe: ee.List(fe.get('sample_pts')).get(0) )
    
    # balance the size of choose_one sample [built = non-built]
    bulit = choose_one.filterMetadata('Built','equals',1)
    non_bulit = choose_one.filterMetadata('Built','equals',0)\
                          .randomColumn()\
                          .limit(bulit.size().getInfo())
    
    # merge to get the hold_out sample
    hold_out = bulit.merge(non_bulit)  
    
    #_____________________________2: exclude the selected points________________________
    
    # using the invert join for exclusion
    invertedJoin = ee.Join.inverted()
    hold_in = invertedJoin.apply(sample_pt, hold_out, spatialFilter)
    
    
    #_____________________________3: download the csv to local drive & gee-asset_______________
    
    # construct export names
    name_hold_out = f'Grid_select_{region_en}_{year}'
    name_hold_in  = f'Training_sample_{region_en}_{year}' 

    # downloding
    for table_name,pts in zip([name_hold_out,name_hold_in],[hold_out,hold_in]):
        
#         # to loacl drive
#         ulr = pts.getDownloadURL('csv') 
#         urllib.request.urlretrieve(ulr, f'./Data/{table_name}.csv')
        
        # to GEE-asset
        task = ee.batch.Export.table.toAsset(collection=pts,
                                             description=table_name,
                                             assetId=f'{export_path}/{table_name}')
        
        task.start()
        
    # print out the process
    print(f'{region_en}_{year} downloaded!')

dongbei_1990_1992 downloaded!


KeyboardInterrupt: 